# This program prepares the selected data from 2007 8th Grade Cohort Longitudinal Study for mapping

### Begin by downloading the Cohort Workbook from [the THECB website](http://www.txhighereddata.org/index.cfm?objectId=F2CBE4A0-C90B-11E5-8D610050560100A9). 

The selected data focuses on target populations from the Texas Higher Education Strategic Plan. The target populations examined here are African American students, particularly African american male students, and Hispanic students.

### Target populations are examined by TEA Region which is how the data is presented by the THECB cohort workbook


In [1]:
import pandas as pd
import requests
import zipfile
import arcpy
import io
import os

arcpy.env.overwriteOutput = True
pd.options.display.max_rows = 10

### Start by downloading the 2007 cohort workbook from the THECB website

The cohort workbooks are available at: http://www.txhighereddata.org/index.cfm?objectId=F2CBE4A0-C90B-11E5-8D610050560100A9

Save the workbook in 'Data\8th Grade FY2007 Cohort Workbook.xlsx'

### First get the Gender by Ethnicity Data

In [2]:
xl = pd.read_excel('Data\8th Grade FY2007 Cohort Workbook.xlsx', sheet_name='TEA by Gender by Ethnicity', header=None, index_col=None, skiprows=6)

#Keep the columns I need
xl2=xl[[0,1,2,3,4,17,18,21,22]]

#Drop the rows I don't need
GenEth=xl2[:160]
GenEth.columns=['TEAReg','RegName','Gender','Eth', 'CohoN', 'nEnr', 'pEnr', 'nComp', 'pComp']

#Make Dataset just of African American Males (60x30TX target popultation - this wasn't used for the maps)
AAmales=GenEth.loc[(GenEth['Eth']=='African American') & (GenEth['Gender']=='Male')].copy() #copy to avoid chained indexing
AAmales=AAmales.drop(['Gender','Eth'], axis=1) #Keep the columns I need
AAmales.columns=['TEAReg','RegName','AAmCoho', 'AAmnEnr', 'AAmpEnr', 'AAmnComp', 'AAmpComp']
AAmales['AAmpEnr']=100*AAmales['AAmpEnr']
AAmales['AAmpComp']=100*AAmales['AAmpComp']

print(AAmales)
#AAmales.to_csv('AAmales.csv', index=False)

    TEAReg         RegName  AAmCoho  AAmnEnr    AAmpEnr  AAmnComp   AAmpComp
4        1        Edinburg     27.0     16.0  59.259259       4.0  14.814815
12       2  Corpus Christi    160.0     70.0  43.750000      16.0  10.000000
20       3        Victoria    237.0     98.0  41.350211      31.0  13.080169
28       4         Houston   8646.0   4358.0  50.404811     922.0  10.663891
36       5        Beaumont    949.0    429.0  45.205479      73.0   7.692308
..     ...             ...      ...      ...        ...       ...        ...
124     16        Amarillo    178.0     80.0  44.943820      14.0   7.865169
132     17         Lubbock    252.0     95.0  37.698413      16.0   6.349206
140     18         Midland    171.0     58.0  33.918129      14.0   8.187135
148     19         El Paso    204.0     93.0  45.588235      21.0  10.294118
156     20     San Antonio   1070.0    513.0  47.943925     118.0  11.028037

[20 rows x 7 columns]


### African American and Hispanic totals by region. 
For this, we'll collapse on ethicity to remove gender.

In [3]:
#Keep Hispanic and African American counts, collapse to remove gender, and then recalculate percents 
EthCounts=GenEth.drop(GenEth.columns[[2,6,8]], axis=1) #axis=0 for rows, axis=1 for columns
#AA_Hisp=EthCounts.loc[EthCounts['Eth'].isin(['African American', 'Hispanic'])]
#AA_Hisp_collapsed=AA_Hisp.groupby(["TEAReg", "RegName","Eth"]).sum()


#Make African American Group
AAtemp=EthCounts.loc[EthCounts['Eth']=='African American'].copy() #copy to avoid chained indexing
AA=AAtemp.groupby(["TEAReg", "RegName","Eth"], as_index=False).sum()
AA['AApEnr']=100*AA['nEnr']/AA['CohoN']
AA['AApComp']=100*AA['nComp']/AA['CohoN']
AA=AA.drop(['Eth'], axis=1) #Keep the columns I need
AA.columns=['TEAReg','RegName','AACoho', 'AAnEnr','AAnComp','AApEnr','AApComp']

#Make Hispanic Group
Hisptemp=EthCounts.loc[EthCounts['Eth']=='Hispanic'].copy() #copy to avoid chained indexing
Hisp=Hisptemp.groupby(["TEAReg", "RegName","Eth"], as_index=False).sum()
Hisp['HispEnr']=100*Hisp['nEnr']/Hisp['CohoN']
Hisp['HispComp']=100*Hisp['nComp']/Hisp['CohoN']
Hisp=Hisp.drop(['Eth'], axis=1) #Keep the columns I need
Hisp.columns=['TEAReg','RegName','HisCoho', 'HisnEnr','HisnComp','HispEnr','HispComp']


#print(AA_Hisp_collapsed)
print(AA)
print(Hisp)
#AA.to_csv('AA.csv')
#Hisp.to_csv('Hisp.csv')

    TEAReg         RegName   AACoho  AAnEnr  AAnComp     AApEnr    AApComp
0        1        Edinburg     60.0    35.0     12.0  58.333333  20.000000
1        2  Corpus Christi    306.0   151.0     40.0  49.346405  13.071895
2        3        Victoria    447.0   222.0     71.0  49.664430  15.883669
3        4         Houston  17218.0  9734.0   2546.0  56.533860  14.786851
4        5        Beaumont   1813.0   957.0    204.0  52.785438  11.252068
..     ...             ...      ...     ...      ...        ...        ...
15      16        Amarillo    364.0   184.0     38.0  50.549451  10.439560
16      17         Lubbock    480.0   202.0     31.0  42.083333   6.458333
17      18         Midland    343.0   143.0     39.0  41.690962  11.370262
18      19         El Paso    380.0   170.0     42.0  44.736842  11.052632
19      20     San Antonio   2074.0  1095.0    342.0  52.796528  16.489875

[20 rows x 7 columns]
    TEAReg         RegName  HisCoho  HisnEnr  HisnComp    HispEnr   HispComp


### Male enrollment and completion by region. 
For this, we'll collapse on gender and remove ethnicity.

In [4]:
#Get total male counts by region, collape on gender, counts only.
GenCounts=GenEth.drop(GenEth.columns[[3,6,8]], axis=1) #axis=0 for rows, axis=1 for columns
Allmalestemp=GenCounts.loc[GenCounts['Gender']=='Male'].copy() #copy to avoid chained indexing
Allmales=Allmalestemp.groupby(["TEAReg", "RegName"], as_index=False).sum().copy()
Allmales['AllmpEnr']=100*Allmales['nEnr']/Allmales['CohoN']
Allmales['AllmpComp']=100*Allmales['nComp']/Allmales['CohoN']
Allmales.columns=['TEAReg', 'RegName','TotmCoho', 'TotmnEnr','TotmnComp','TotmpEnr','TotmpComp']


print(Allmales)
#Allmales.to_csv('Allmales.csv')

    TEAReg         RegName  TotmCoho  TotmnEnr  TotmnComp   TotmpEnr  \
0        1        Edinburg   13488.0    7135.0     2411.0  52.898873   
1        2  Corpus Christi    4113.0    1857.0      601.0  45.149526   
2        3        Victoria    2144.0    1008.0      435.0  47.014925   
3        4         Houston   37900.0   19325.0     6852.0  50.989446   
4        5        Beaumont    3199.0    1503.0      525.0  46.983432   
..     ...             ...       ...       ...        ...        ...   
15      16        Amarillo    2987.0    1443.0      515.0  48.309340   
16      17         Lubbock    2899.0    1361.0      498.0  46.947223   
17      18         Midland    3005.0    1311.0      433.0  43.627288   
18      19         El Paso    6775.0    3673.0     1041.0  54.214022   
19      20     San Antonio   14188.0    6711.0     2407.0  47.300536   

    TotmpComp  
0   17.875148  
1   14.612205  
2   20.289179  
3   18.079156  
4   16.411379  
..        ...  
15  17.241379  
16  17.

### Get Economic Disadvantaged student data by region. 

In [5]:
xlEcon = pd.read_excel('Data\8th Grade FY2007 Cohort Workbook.xlsx', sheet_name='TEA Region by Eco', header=None, index_col=None, skiprows=6)

#Keep the columns I need
xlEcon2=xlEcon[[0,1,2,3,16,17,20,21]]
EconTemp=xlEcon2.loc[xlEcon2[2]=='Economically Disadvantaged'].copy()

EconTemp2=EconTemp.drop([2], axis=1).copy()

#Get Region Totals and drop the rows I don't need
Econ=EconTemp2[:20].copy()
Econ.columns=['TEAReg','RegName','EcoCoho', 'EconEnr', 'EcopEnr', 'EconComp', 'EcopComp']

Econ['EcopEnr']=100*Econ['EcopEnr']
Econ['EcopComp']=100*Econ['EcopComp']
print(Econ)

   TEAReg         RegName  EcoCoho  EconEnr    EcopEnr  EconComp   EcopComp
1       1        Edinburg  22752.0  12222.0  53.718354    4377.0  19.237869
3       2  Corpus Christi   4635.0   1816.0  39.180151     460.0   9.924488
5       3        Victoria   2153.0    833.0  38.690200     250.0  11.611705
7       4         Houston  37985.0  16583.0  43.656707    4718.0  12.420692
9       5        Beaumont   3056.0   1260.0  41.230366     304.0   9.947644
..    ...             ...      ...      ...        ...       ...        ...
31     16        Amarillo   3005.0   1280.0  42.595674     393.0  13.078203
33     17         Lubbock   3198.0   1211.0  37.867417     329.0  10.287680
35     18         Midland   3039.0   1109.0  36.492267     296.0   9.740046
37     19         El Paso   9956.0   5500.0  55.243070    1683.0  16.904379
39     20     San Antonio  16482.0   7128.0  43.247179    2075.0  12.589492

[20 rows x 7 columns]


### Here we get overall totals by region for comparison

In [6]:
xl = pd.read_excel('Data\8th Grade FY2007 Cohort Workbook.xlsx', sheet_name='Summary', header=None, index_col=None, skiprows=16)

#Keep the columns I need
xl2=xl[[0,1,2,15,16,19,20]]

#Get Region Totals and drop the rows I don't need
RegTotals=xl2[:20].copy()
RegTotals.columns=['TEAReg','RegName','TotCoho', 'TotnEnr', 'TotpEnr', 'TotnComp', 'TotpComp']

RegTotals['TotpEnr']=100*RegTotals['TotpEnr']
RegTotals['TotpComp']=100*RegTotals['TotpComp']

print(RegTotals)
#RegTotals.to_csv('RegTotals.csv', index=False)

   TEAReg         RegName  TotCoho  TotnEnr    TotpEnr  TotnComp   TotpComp
0       1        Edinburg  26581.0  15042.0  56.589293    5754.0  21.647041
1       2  Corpus Christi   7912.0   3996.0  50.505561    1429.0  18.061173
2       3        Victoria   4109.0   2197.0  53.467997     995.0  24.215138
3       4         Houston  74398.0  40848.0  54.904702   16498.0  22.175327
4       5        Beaumont   6094.0   3224.0  52.904496    1235.0  20.265835
..    ...             ...      ...      ...        ...       ...        ...
15     16        Amarillo   5830.0   3157.0  54.150943    1278.0  21.921098
16     17         Lubbock   5639.0   2855.0  50.629544    1150.0  20.393687
17     18         Midland   5880.0   2837.0  48.248299    1021.0  17.363946
18     19         El Paso  13214.0   7716.0  58.392614    2605.0  19.713940
19     20     San Antonio  27421.0  14194.0  51.763247    5645.0  20.586412

[20 rows x 7 columns]


### Now get statewide Cohort totals for Hispanics and African Americans
(not used for the maps)

In [7]:
xl = pd.read_excel('Data\8th Grade FY2007 Cohort Workbook.xlsx', sheet_name='Summary', header=None, index_col=None, skiprows=38)

#Keep the columns I need
xl2=xl[[0,1,2]]

#Get Region Totals and drop the rows I don't need
StatewideCohortTotals=xl2[:8]
StatewideCohortTotals.columns=['Gender','Eth','Cohort']

#Get African American and Hispanic Statewide Cohort Totals
StatewideCohortTotals=StatewideCohortTotals.groupby(["Eth"]).sum().copy()

print(StatewideCohortTotals)
#StatewideCohortTotals.to_csv('StatewideCohortTotals.csv', index=False)

                    Cohort
Eth                       
African American   50684.0
Hispanic          149548.0
Others             11896.0
White             126214.0


### Now get statewide Cohort totals for Econ Disadvantage

In [8]:
xl = pd.read_excel('Data\8th Grade FY2007 Cohort Workbook.xlsx', sheet_name='Summary', header=None, index_col=None, skiprows=52)

#Keep the columns I need
xl2=xl[[0,1,2]]

#Get Region Totals and drop the rows I don't need
StatewideCohortEcon=xl2[:4]
StatewideCohortEcon.columns=['Eco','Eth','Cohort']

#Get African American and Hispanic Statewide Cohort Totals
StatewideCohortEcon=StatewideCohortEcon.groupby(["Eco"]).sum().copy()

print(StatewideCohortEcon)

                              Cohort
Eco                                 
Economically Disadvantaged  179535.0


### And now merge the tables

In [9]:
#Combine into one table
All=pd.merge(AA, Hisp,on=['TEAReg', 'RegName']).copy()
All=pd.merge(All, Allmales,on=['TEAReg', 'RegName']).copy()
All=pd.merge(All, RegTotals,on=['TEAReg', 'RegName']).copy()
All=pd.merge(All, Econ,on=['TEAReg', 'RegName']).copy()


#Calculate Hisp, AA, and Econ % of statewide cohort for each TEA Region
All['AATXCoho']=StatewideCohortTotals.loc['African American','Cohort']
All['HisTXCoho']=StatewideCohortTotals.loc['Hispanic','Cohort']
All['EcoTXCoho']=StatewideCohortEcon.loc['Economically Disadvantaged','Cohort']
All['AApTXCoho']=100*All['AACoho']/All['AATXCoho']
All['HispTXCoho']=100*All['HisCoho']/All['HisTXCoho']
All['EcopTXCoho']=100*All['EcoCoho']/All['EcoTXCoho']


#Calculate % point differences for AA/Hisp/AAmales/Eco enrollmnet and completion rates from total
All['AAEnrpDi']=All['AApEnr']-All['TotpEnr']
All['HisEnrpDi']=All['HispEnr']-All['TotpEnr']
All['AAmEnrpDi']=All['AAmpEnr']-All['TotmpEnr'] #AA males only vs all males (not used)
All['TotmEnrpDi']=All['TotmpEnr']-All['TotpEnr'] #all males
All['EcoEnrpDi']=All['EcopEnr']-All['TotpEnr']
All['AAComppDi']=All['AApComp']-All['TotpComp']
All['HisComppDi']=All['HispComp']-All['TotpComp']
All['AAmComppDi']=All['AAmpComp']-All['TotmpComp'] #AA males only vs all males (not used)
All['TotmCompDi']=All['TotmpComp']-All['TotpComp'] #all males
All['EcoComppDi']=All['EcopComp']-All['TotpComp']

#Drop unneeded variables
Final=All.drop(['HisTXCoho','AATXCoho', 'EcoTXCoho'], axis=1).copy() #Keep the columns I need


#Make perc of total for AA, Hisp, Eco, and AA_males
Final['AApCoho']=100*All['AACoho']/All['TotCoho']
Final['HispCoho']=100*All['HisCoho']/All['TotCoho']
Final['AAmpCoho']=100*All['AAmCoho']/All['TotmCoho']
Final['EcopCoho']=100*All['EcoCoho']/All['TotCoho']

#Make variables with "_" suffix. They will have zero decmals and be used as symbol layers
Final['TotpEnr_']=Final['TotpEnr']
Final['TotpComp_']=Final['TotpComp'] 
Final['AApCoho_']=Final['AApCoho']
Final['AAmpCoho_']=Final['AAmpCoho']
Final['HispCoho_']=Final['HispCoho']
Final['EcopCoho_']=Final['EcopCoho']
Final['AAComppD_']=Final['AAComppDi']
Final['AAmComppD_']=Final['AAmComppDi']
Final['HisComppD_']=Final['HisComppDi']
Final['EcoComppD_']=Final['EcoComppDi']
Final['AAEnrpD_']=Final['AAEnrpDi']
Final['AAmEnrpD_']=Final['AAmEnrpDi']
Final['HisEnrpD_']=Final['HisEnrpDi']
Final['EcoEnrpD_']=Final['EcoEnrpDi']
Final['TotmEnrpD_']=Final['TotmEnrpDi']
Final['TotmCompD_']=Final['TotmCompDi']


#set percentages to have just one decimal place
Processed = Final.round({'AApEnr': 1, 'AApComp': 1, 'AAmpEnr': 1, 'AAmpComp': 1, 
             'HispEnr': 1, 'HispComp': 1, 'TotmpEnr': 1, 'TotmpComp': 1, 
             'TotpEnr': 1, 'TotpComp': 1, 'AApTXCoho': 1, 'AAEnrpDi': 1, 
             'HisEnrpDi': 1, 'AAmEnrpDi': 1, 'AAComppDi': 1, 'HisComppDi': 
             1, 'AAmComppDi': 1, 'AApCoho': 1, 'HispCoho': 1, 'AAmpCoho': 1,
             'EcopEnr': 1, 'EcopComp': 1, 'EcoEnrpDi': 1, 'EcoComppDi': 1, 
            'EcopTXCoho': 1,'HispTXCoho': 1, 'EcopCoho':1, 'TotmEnrpDi':1, 'TotmCompDi':1, 'TotpEnr_':0, 
            'TotpComp_':0, 'AApCoho_':0, 'AAmpCoho_':0, 'HispCoho_':0, 'EcopCoho_':0, 
            'AAComppD_':0, 'AAmComppD_':0, 'HisComppD_':0, 'EcoComppD_':0,
            'AAEnrpD_':0, 'AAmEnrpD_':0, 'HisEnrpD_':0, 'EcoEnrpD_':0, 'TotmEnrpD_':0, 'TotmCompD_':0}).copy()

Processed.to_csv('ProcessedData.csv', index=False)
print(Processed)

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

# The rest of the code prepares the shapefiles for mapping.

### We'll need:
    
* Polygons for TEA Regions [available from TEA](http://schoolsdata2-tea-texas.opendata.arcgis.com)
* Centroids (points) for TEA Regions
* An outline of the State of Texas
    
    

In [ ]:
#get TEARegion file and unzip
URL=requests.get('http://opendata.arcgis.com/datasets/12142ff8beec4a1797334c9c41ba7b18_0.zip')
zippedRegions=zipfile.ZipFile(io.BytesIO(URL.content))
zippedRegions.extractall('Data/rawESC_Regions')

#Delete unnecessary fields
arcpy.DeleteField_management("Data/rawESC_Regions/ESC_Regions.shp", ['SHAPE_Area'])

In [ ]:
#get State of Texas file and unzip
URLtexas=requests.get('http://www2.census.gov/geo/tiger/GENZ2016/shp/cb_2016_us_state_5m.zip')
zippedState=zipfile.ZipFile(io.BytesIO(URLtexas.content))
zippedState.extractall('Data/TexasOutline')

#Delete unnecessary fields
arcpy.DeleteField_management("Data/TexasOutline/cb_2016_us_state_5m.shp", 
                             ["sTATENS", "AFFGEOID", "STUSPS", 'NAME', 'LSAD', 'ALAND', 'AWATER'])


arcpy.MakeFeatureLayer_management ("Data/TexasOutline/cb_2016_us_state_5m.shp", "TexasOutline", "STATEFP='48'")



In [ ]:
# Create a File Geodatabase and copy shapefile data
# uncomment the following line the first time code is run
arcpy.CreateFileGDB_management('Data',"Cohort.gdb")

arcpy.FeatureClassToGeodatabase_conversion('Data/rawESC_Regions/ESC_Regions.shp', 'Data/Cohort.gdb')

#List fields in dataset
fields = arcpy.ListFields('Data/Cohort.gdb/ESC_Regions')

for field in fields:
    print("{0} is a type of {1} with a length of {2}"
          .format(field.name, field.type, field.length))

In [ ]:
#Add Cohort data to GeoDataBase
arcpy.TableToTable_conversion('ProcessedData.csv', 'Data/Cohort.gdb', 'CohortData')

#Merge Cohort Data to TEA Region Polygons
arcpy.JoinField_management('Data/Cohort.gdb/ESC_Regions', 'OBJECTID','Data/Cohort.gdb/CohortData', 'TEAReg')

In [ ]:
#Make folder if it doesn't exist
if not os.path.exists('Data/FinalShapefiles'):
    os.makedirs('Data/FinalShapefiles')
    
#Export merged TEARegions with Cohort data to shapefile
arcpy.FeatureClassToShapefile_conversion ('Data/Cohort.gdb/ESC_Regions', 'Data/FinalShapefiles')

#Export TexasOutline to shapefile
arcpy.FeatureClassToShapefile_conversion ('TexasOutline', 'Data/FinalShapefiles')

### Now make the centrids for the TEA Regions

(Requires the advanced license)

In [ ]:
#  Set local variables
inFeatures = "Data/Cohort.gdb/ESC_Regions"
outFeatureClass = "Data/Cohort.gdb/ESC_Points"

# Use FeatureToPoint function to find a point inside each park
arcpy.FeatureToPoint_management(inFeatures, outFeatureClass)

In [ ]:
#Export merged TEARegion Points to shapefile
arcpy.FeatureClassToShapefile_conversion ('Data/Cohort.gdb/ESC_Points', 'Data/FinalShapefiles')

### Now, go to linux and use the GDAL to convert shapefiles to geojson. Then use the Tippecanoe tool to make .MBtiles

I used the following commands:

* ogr2ogr -f GeoJSON CohortTEARegionPolys.json Data/FinalShapefiles/ESC_Regions.shp -progress
* ogr2ogr -f GeoJSON TexasOutline.json Data/FinalShapefiles/TexasOutline.shp -progress
* ogr2ogr -f GeoJSON CohortTEARegionPoints.json Data/FinalShapefiles/ESC_Points.shp -progress
* tippecanoe --output=8thGradeCohort2007TEARegionData.mbtiles CohortTEARegionPoints.json CohortTEARegionPolys.json TexasOutline.json -r1 --drop-fraction-as-needed  --simplification=9 --maximum-zoom=15 --minimum-zoom=3 --exclude=OBJECTID_1 --detect-shared-borders